In [1]:
# Create a virtual environment named 'env'
!python -m venv env

# Activate the virtual environment
# For Windows
!.\env\Scripts\activate

In [2]:
# Install packages from requirements.txt
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
You should consider upgrading via the 'C:\Users\suyog\OneDrive\Desktop\Projects\DA_AI_Agent\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
# List installed packages to verify installation
!pip list

Package                      Version
---------------------------- ------------
absl-py                      2.1.0
annotated-types              0.7.0
anyio                        4.8.0
asttokens                    3.0.0
astunparse                   1.6.3
bleach                       6.2.0
cachetools                   5.5.1
certifi                      2025.1.31
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.1
cycler                       0.12.1
debugpy                      1.8.12
decorator                    5.1.1
distro                       1.9.0
docstring_parser             0.16
exceptiongroup               1.2.2
executing                    2.2.0
fastapi                      0.115.8
filelock                     3.17.0
flatbuffers                  25.2.10
fonttools                    4.56.0
fsspec                       2025.2.0
gast                         0.4

You should consider upgrading via the 'C:\Users\suyog\OneDrive\Desktop\Projects\DA_AI_Agent\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
import torch
from dotenv import load_dotenv
import os
import pandas as pd
import re
from kaggle.api.kaggle_api_extended import KaggleApi
from phi.agent import Agent
from phi.model.groq import Groq
import pandasql as psql 

# Load environment variables
load_dotenv()

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.5.1+cu121
CUDA Available: True
CUDA Version: 12.1
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
# verify GPU perfrmance
import torch
x = torch.rand(10000, 10000).cuda()
print(x.device)

cuda:0


-------------------------- AI Agent for SQL query builder --------------------------

------------------------------------ SET UP KAGGLE API  ---------------------------

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

username = os.getenv("KAGGLE_USERNAME")
key = os.getenv("KAGGLE_KEY")
from kaggle.api.kaggle_api_extended import KaggleApi

print("Kaggle Username:", username)

# Initialize Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

print("Kaggle API authenticated successfully!")


print("KAGGLE_USERNAME:", os.getenv("KAGGLE_USERNAME"))
print("KAGGLE_KEY:", os.getenv("KAGGLE_KEY"))


Kaggle Username: suyog2
Kaggle API authenticated successfully!
KAGGLE_USERNAME: suyog2
KAGGLE_KEY: 257621bd068e929792413446b2aac650


In [8]:
datasets = api.dataset_list(search="sales data")
for dataset in datasets:
    print(dataset.ref)


syedanwarafridi/vehicle-sales-data
kyanyoga/sample-sales-data
ddosad/auto-sales-data
milanzdravkovic/pharma-sales-data
yapwh1208/supermarket-sales-data
karkavelrajaj/amazon-sales-dataset
thedevastator/unlock-profits-with-e-commerce-sales-data
rohitsahoo/sales-forecasting
ksabishek/product-sales-data
suraj520/car-sales-data
berkayalan/retail-sales-data
samruddhi4040/online-sales-data
shreyanshverma27/online-sales-dataset-popular-marketplace-data
ronnykym/online-store-sales-data
willianoliveiragibin/market-sales-data
talhabu/us-regional-sales-data
harrimansaragih/dummy-advertising-and-sales-data
mehmettahiraslan/customer-shopping-dataset
mohammadtalib786/retail-sales-dataset
lokeshparab/amazon-products-dataset


--------------------------------- Download specific dataset from Kaggle  -------------- 

In [ ]:

# Get Kaggle credentials from environment variables
username = os.getenv("KAGGLE_USERNAME")
key = os.getenv("KAGGLE_KEY")

# Initialize Kaggle API
api = KaggleApi()
api.authenticate()

# Define dataset name
dataset_name = "thedevastator/unlock-profits-with-e-commerce-sales-data"  # Adjust the dataset name as needed

# Download dataset
api.dataset_download_files(dataset_name, path="./data", unzip=True)


Dataset URL: https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data


---------------------------------------- Import files into pandas dataframes ----------------------

In [7]:
data_folder = "./data/"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# Dictionary to store DataFrames
dataframes = {}

# Loop through CSV files and read each one into a DataFrame
for file in csv_files:
    file_path = os.path.join(data_folder, file)
    df_name = file.replace(".csv", "")  # Use filename as DataFrame key
    dataframes[df_name] = pd.read_csv(file_path)
    print(f"Loaded {file} into DataFrame: {df_name}")

# Preview the first few rows of each DataFrame
for name, df in dataframes.items():
    print(f"\nPreview of {name} DataFrame:")
    print(df.head())


C:\Users\suyog\AppData\Local\Temp\ipykernel_1576\3750667273.py:13: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[df_name] = pd.read_csv(file_path)


Loaded Amazon Sale Report.csv into DataFrame: Amazon Sale Report
Loaded Cloud Warehouse Compersion Chart.csv into DataFrame: Cloud Warehouse Compersion Chart
Loaded Expense IIGF.csv into DataFrame: Expense IIGF
Loaded International sale Report.csv into DataFrame: International sale Report
Loaded May-2022.csv into DataFrame: May-2022
Loaded P  L March 2021.csv into DataFrame: P  L March 2021
Loaded Sale Report.csv into DataFrame: Sale Report

Preview of Amazon Sale Report DataFrame:
   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  

In [ ]:
# Initialize AI SQL Agent
sql_agent = Agent(model=Groq(id="llama-3.3-70b-versatile"))

# Function to generate SQL query with JOINs
def generate_sql_query(natural_language_query, table_schemas):
    # Format table schemas for AI input
    schema_info = "\n".join([f"Table `{name}`: Columns -> {', '.join(columns)}" for name, columns in table_schemas.items()])
    
    prompt = f"""
    You are an expert SQL assistant. Convert the following natural language request into a valid SQL query. 
    The database contains multiple tables with relationships inferred from common columns.

    Information about the dataset:
    {schema_info}

    The user request is: "{natural_language_query}"
    
    Generate an optimized SQL query with proper JOIN operations if needed.
    """
    
    response = sql_agent.run(prompt)
    return response.content  # Extract generated SQL


# Define the table schemas
table_schemas = {
    'Cloud Warehouse Compersion Chart': ['Shiprocket', 'INCREFF'],
    'Sale Report': ['SKU Code', 'Design No.', 'Stock', 'Category', 'Size', 'Color'],
    'P & L March 2021': ['Category', 'Sku', 'Catalog', 'Weight', 'TP 1', 'TP 2', 'MRP Old', 'Final MRP Old', 'Ajio MRP', 'Amazon MRP', 'Amazon FBA MRP', 'Flipkart MRP', 'Limeroad MRP', 'Myntra MRP', 'Paytm MRP', 'Snapdeal MRP'],
    'May-2022': ['Sku', 'Catalog', 'Category', 'Weight', 'MRP Old', 'Final MRP Old', 'Ajio MRP', 'Amazon MRP', 'Amazon FBA MRP', 'Flipkart MRP', 'Limeroad MRP', 'Myntra MRP', 'Paytm MRP', 'Snapdeal MRP'],
    'Amazon Sale Report': ['Category', 'Size', 'Date', 'Status', 'Fulfilment', 'Style', 'SKU', 'ASIN', 'Courier Status', 'Qty', 'Amount', 'B2B', 'Currency'],
    'International sale Report': ['Style', 'SKU', 'Size', 'DATE', 'Months', 'CUSTOMER', 'PCS', 'RATE', 'GROSS AMT'],
    'Expense IIGF': ['Recived Amount']
}


Generated SQL Query:
SELECT 
    DATE_FORMAT(a.Date, '%Y-%m') as Month,
    a.Fulfilment,
    SUM(a.Amount) as Total_Amount,
    COUNT(DISTINCT a.Sku) as Total_SKU
FROM 
    `Amazon Sale Report` a
GROUP BY 
    DATE_FORMAT(a.Date, '%Y-%m'),
    a.Fulfilment
ORDER BY 
    DATE_FORMAT(a.Date, '%Y-%m'),
    a.Fulfilment;


In [22]:
# Example user request that requires a JOIN
user_query = "Get total sales for each month by Method of fulfilment, including their Amount of the sale and count of SKU."
sql_query = generate_sql_query(user_query, table_schemas)

# Extract only the SQL query part using regex to remove any extraneous text
sql_query = re.search(r"(?s)```sql(.*?)```", sql_query).group(1).strip()

# Replace EXTRACT with strftime for SQLite compatibility
sql_query = sql_query.replace("EXTRACT(MONTH FROM ASR.Date)", "strftime('%m', ASR.Date)")

print("\nGenerated SQL Query:")
print(sql_query)


Generated SQL Query:
SELECT 
    EXTRACT(MONTH FROM `Date`) AS SalesMonth,
    Fulfilment AS MethodOfFulfilment,
    SUM(Amount) AS TotalSalesAmount,
    COUNT(DISTINCT SKU) AS CountOfSKU
FROM 
    `Amazon Sale Report`
GROUP BY 
    EXTRACT(MONTH FROM `Date`), Fulfilment
ORDER BY 
    SalesMonth, MethodOfFulfilment;
